In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

#Scale features

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))


y_train = y_train.view(y_train.shape[0],1) # to transform it in style [ [1],[1],[1] ] FROM [1,0,1,1]
y_test = y_test.view(y_test.shape[0],1)



In [ ]:
#Design model

# f = w*x + b, sigmoid at the end

class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features,1)
  
  def forward(self,x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

model = LogisticRegression(n_features)

In [ ]:
#Loss and optimizer

learning_rate = 0.001

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
#Training loop

num_epochs = 100
for epoch in range(num_epochs):
  #forward pass and loss

  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  #backward pass
  loss.backward()

  #update weights
  optimizer.step()

  # zero gradients
  optimizer.zero_grad()

  if (epoch+1)%10 == 0:
    print(f'epoch {epoch+1}, loss = {loss.item():.4f}')


with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_classes = y_predicted.round()
  acc = y_predicted_classes.eq(y_test).sum() / float(y_test.shape[0])
  print(f"accuracy = {acc:.4f}")


epoch 10, loss = 0.4436
epoch 20, loss = 0.4383
epoch 30, loss = 0.4330
epoch 40, loss = 0.4280
epoch 50, loss = 0.4231
epoch 60, loss = 0.4184
epoch 70, loss = 0.4139
epoch 80, loss = 0.4095
epoch 90, loss = 0.4052
epoch 100, loss = 0.4010
accuracy = 0.9561
